# Notebook to show the validate event pre-processing and formatting

In [2]:
%matplotlib inline
from importlib import reload
from pathlib import Path

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid', palette='muted')

from types import SimpleNamespace

from TreeMazeAnalyses2.Analyses import subject_info as si
import TreeMazeAnalyses2.Analyses.tree_maze_functions as tmf
import Analyses.spatial_functions as spatial_funcs
import Utils.filter_functions as filter_funcs

from pykalman import KalmanFilter
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)


In [3]:
subject = 'Li'
session = 'Li_T3g_070618'

In [4]:
si = reload(si)
overwrite=True
subject_info = si.SubjectInfo(subject, overwrite=overwrite)

Error Computing Cluster Stats for Li_OF_052818


In [5]:
# load a session
si = reload(si)
session_info = si.SubjectSessionInfo(subject, session)

In [19]:
tmf = reload(tmf)
b = tmf.BehaviorData(session_info, overwrite=True)

In [20]:
b.trial_table.head(10)

,t0,tE,dur,cue,dec,correct,long,goal,sw,vsw
0,101,523,422,L,L,1,1,4,0,0
1,890,1332,441,R,L,0,-1,-1,1,1
2,1855,2361,506,L,R,0,-1,-1,1,0
3,2824,3539,714,R,L,0,-1,-1,1,0
4,3759,4434,674,L,L,1,1,4,1,0
5,4608,4984,376,L,L,1,0,3,0,0
6,5326,5620,294,L,L,1,0,3,0,0
7,5961,8598,2637,R,L,0,-1,-1,1,1
8,8927,9264,337,R,R,1,1,1,0,0
9,9636,10018,381,R,R,1,1,1,0,0


In [61]:
df = b.trial_table
perf = pd.DataFrame(index=[0])
perf['n_trials'] = df.shape[0]
perf['n_sw_trials'] = df.sw.sum()
perf['n_vsw_trials'] = df.vsw.sum()
perf['pct_correct'] = df.correct.mean()
perf['pct_sw_correct'] = (df.sw & df.correct).sum()/perf.n_sw_trials
perf['pct_vsw_correct'] = (df.vsw & df.correct).sum()/perf.n_vsw_trials
perf['n_L_trials'] = (df.cue=='L').sum()
perf['n_R_trials'] = (df.cue=='R').sum()
perf['pct_L_correct'] = ((df.cue=='L') & df.correct).sum() / perf.n_L_trials
perf['pct_R_correct'] = ((df.cue=='R') & df.correct).sum() / perf.n_R_trials
perf

,n_trials,n_sw_trials,n_vsw_trials,pct_correct,pct_sw_correct,pct_vsw_correct,n_L_trials,n_R_trials,pct_L_correct,pct_R_correct
0,183,95,62,0.622951,0.578947,0.645161,89,94,0.561798,0.680851


In [60]:
perf = pd.DataFrame(index=[0])
perf['n_trials'] = df.shape[0]
perf

,n_trials
0,183


In [25]:
b.trial_table.head()

,t0,tE,dur,cue,dec,corr
0,24598,25021,422,L,L,1
1,25388,25830,441,R,L,0
2,26352,26859,506,L,R,0
3,27322,28036,714,R,L,0
4,28257,28932,674,L,L,1


In [44]:
df = b.trial_table.copy()
df['sw'] = 0
df.loc[1:, 'sw'] = b.trial_table['cue'][:-1].values!=b.trial_table['cue'][1:].values


df['vsw'] = 0
df.loc[1:, 'vsw'] = (df['corr'][:-1].values) & (df['sw'][1:].values)

df = df.astype({'sw':'int', 'vsw':'int'})
df.head(20)

,t0,tE,dur,cue,dec,corr,sw,vsw
0,24598,25021,422,L,L,1,0,0
1,25388,25830,441,R,L,0,1,1
2,26352,26859,506,L,R,0,1,0
3,27322,28036,714,R,L,0,1,0
4,28257,28932,674,L,L,1,1,0
5,29105,29481,376,L,L,1,0,0
6,29823,30118,294,L,L,1,0,0
7,30458,33096,2637,R,L,0,1,1
8,33424,33762,337,R,R,1,0,0
9,34134,34515,381,R,R,1,0,0


In [46]:
(df.cue=='L')

0       True
1      False
2       True
3      False
4       True
5       True
6       True
7      False
8      False
9      False
10     False
11      True
12      True
13      True
14     False
15     False
16      True
17     False
18     False
19     False
20     False
21      True
22     False
23      True
24     False
25     False
26      True
27     False
28     False
29     False
30      True
31     False
32      True
33      True
34      True
35      True
36      True
37      True
38      True
39     False
40     False
41     False
42      True
43      True
44      True
45      True
46      True
47     False
48      True
49     False
50     False
51     False
52     False
53      True
54      True
55      True
56     False
57     False
58      True
59     False
60      True
61      True
62     False
63     False
64      True
65      True
66      True
67     False
68      True
69     False
70     False
71      True
72     False
73     False
74     False
75     False
76      True

In [34]:
df['vsw'] = 0
df.loc[1:, 'vsw'] = (df['corr'][:-1].values) & (df['sw'][1:].values)

df.head(10)

,t0,tE,dur,cue,dec,corr,switch,v_switch
0,24598,25021,422,L,L,1,0,0
1,25388,25830,441,R,L,0,1,1
2,26352,26859,506,L,R,0,1,0
3,27322,28036,714,R,L,0,1,0
4,28257,28932,674,L,L,1,1,0
5,29105,29481,376,L,L,1,0,0
6,29823,30118,294,L,L,1,0,0
7,30458,33096,2637,R,L,0,1,1
8,33424,33762,337,R,R,1,0,0
9,34134,34515,381,R,R,1,0,0


In [13]:
b.event_table.head()

,event,t0,tE,dur,trial_num,out_bound
0,Start,24497,24497,0,-1,0
1,DE2,24530,24535,5,-1,0
2,DE1,24591,24596,5,-1,0
3,RW1,24591,24615,24,-1,0
4,RD,24591,24615,24,-1,0


In [62]:
a = b.event_durs[b.event_durs.event=='CL'].copy()

In [63]:
a.reset_index(drop='True')

,event,t0,tE,dur,trial_num,out_bound
0,CL,24598,25021,423,0,True
1,CL,26352,26786,434,2,True
2,CL,28257,28932,675,4,True
3,CL,29105,29481,376,5,True
4,CL,29823,30118,295,6,True
5,CL,35608,36070,462,11,True
6,CL,36590,36980,390,12,True
7,CL,37166,37415,249,13,True
8,CL,39166,39480,314,16,True
9,CL,45854,46477,623,21,True


In [85]:
L_trials = b.trial_table[b.trial_table.dec=='L']
L_trials.index.values

array([  0,   1,   3,   4,   5,   6,   7,  10,  12,  13,  14,  16,  17,
        19,  20,  21,  24,  25,  26,  28,  30,  32,  34,  36,  37,  38,
        42,  44,  45,  48,  52,  54,  55,  59,  61,  63,  65,  66,  69,
        74,  76,  78,  82,  86,  87,  90,  95,  96,  97,  99, 100, 103,
       109, 112, 113, 115, 116, 124, 125, 131, 135, 140, 143, 145, 148,
       149, 151, 155, 156, 158, 162, 163, 167, 168, 170, 174, 175, 179,
       181])

In [72]:
a = b.event_durs.loc[b.event_durs.event=='RW2', ['tE', 'trial_num']].copy()

,tE
trial_num,
0,24781
1,25575
2,26548
3,27523
4,28414
5,29275
7,32456
9,34241
10,34971


In [95]:
tmf = reload(tmf)
b = tmf.BehaviorData(session_info)

AttributeError: 'SubjectSessionInfo' object has no attribute 'get_raw_events'